# Normalize CSV columns

Normalize the columns for the eODP CSVs.

In [1]:
import sys
sys.path.append('../scripts/')

import pandas as pd

from normalize_data import (
    normalize_sample_col, 
    normalize_expedition_section_cols, 
    csv_cleanup,
    update_metadata,
    fetch_unique_column_names,
    append_set,
    filter_existing_set,
    normalize_columns,
    add_missing_columns
)

In [2]:
metadata_file = 'cleaned_data/metadata/Lithology_changes.csv'
clean_data_path = 'cleaned_data/Lithology_CSV'

In [2]:
metadata_file = 'cleaned_data/metadata/Micropal_1_changes.csv'
clean_data_path = 'cleaned_data/Micropal_CSV_1'

In [2]:
metadata_file = 'cleaned_data/metadata/Micropal_2_changes.csv'
clean_data_path = 'cleaned_data/Micropal_CSV_2'

## Normalize expedition..section columns 

Read each Lithology CSV to check if expedition..section columns exist. Overwrite existing Lithology CSV if columns need to be added.

### Read metadata file

In [3]:
metadata = pd.read_csv(metadata_file)
metadata.head()

,file,taxon_group
0,356-U1464C_benthic_forams.csv,benthic_forams
1,374_U1523B_benthic_forams.csv,benthic_forams
2,353_U1443A_nannofossils.csv,nannofossils
3,369_U1513D_planktic_forams.csv,planktic_forams
4,371_U1510B_planktic_forams.csv,planktic_forams


In [4]:
metadata.shape

(511, 2)

### Normalize columns

In [5]:
def process_filename(file):
    path = f"{clean_data_path}/{file}"
    content = pd.read_csv(path)
    
    original_cols = content.columns
    content = normalize_expedition_section_cols(content)
    changed = list(original_cols) != list(content.columns)

        
    if changed:
        content = csv_cleanup(content, path)
        content.to_csv(path, index=False)

    return changed
    
change_columns = [process_filename(file) for file in metadata['file']] 


### Update metadata

In [6]:
dict = {"add_expedition_section_cols": change_columns}
new_metadata = update_metadata(metadata, dict)
new_metadata.head()

,file,taxon_group,add_expedition_section_cols
0,356-U1464C_benthic_forams.csv,benthic_forams,True
1,374_U1523B_benthic_forams.csv,benthic_forams,True
2,353_U1443A_nannofossils.csv,nannofossils,True
3,369_U1513D_planktic_forams.csv,planktic_forams,True
4,371_U1510B_planktic_forams.csv,planktic_forams,True


In [7]:
new_metadata.to_csv(metadata_file, index=False)

## Normalize Sample column

Read each Lithology CSV to check if Sample column needs to be updated. Change 'Label ID' to 'Sample'. Add 'Sample' if no 'Sample' or 'Label ID' based on expedition...section columns. Overwrite existing Lithology CSV if Sample column is updated.

### Read metadata file

In [8]:
metadata = pd.read_csv(metadata_file)

In [9]:
metadata.shape

(511, 3)

### Normalize columns

In [ ]:
def process_filename(file):
    path = f"{clean_data_path}/{file}"
    content = pd.read_csv(path)
    
    original_cols = content.columns
    normalize_sample_col(content)
    
    changed = list(original_cols) != list(content.columns)
    
    if changed:
        content = csv_cleanup(content, path)
        content.to_csv(path, index=False)

    return changed
    
change_columns = [process_filename(file) for file in metadata['file']] 

### Update metadata

In [ ]:
dict = {"update_sample_col": change_columns}
new_metadata = update_metadata(metadata, dict)
new_metadata.head()

In [12]:
new_metadata.to_csv(metadata_file, index=False)

## Normalize Top and Bottom columns

Normalize all the Top, Top Depth, Bottom, and Bottom Depth column to have the same names.

In [3]:
metadata = pd.read_csv(metadata_file)

### Get top bottom columns

In [4]:
columns_all = set()

res=[fetch_unique_column_names(f"{clean_data_path}/{file}", columns_all) for file in metadata['file']] 

In [5]:
top_all = set()
top = set()
top_depth = set()

bottom_all = set()
bottom = set()
bottom_depth = set()

append_set(top_all, r".*?top.*?", columns_all)
append_set(top_depth, r"top depth", columns_all)
append_set(top, r"top offset|top ?\[", columns_all)

append_set(bottom_all, r".*?bottom.*?", columns_all)
append_set(bottom_depth, r"bottom depth", columns_all)
append_set(bottom, r"bottom offset|bottom ?\[", columns_all)

In [6]:
top_all

{'Acarinina praetopilensis',
 'Acarinina praetopilensis Blow, 1979',
 'Acarinina pseudotopilensis',
 'Acarinina pseudotopilensis Subbotina, 1953',
 'Acarinina topilensis',
 'Acarinina topilensis (Cushman, 1925)',
 'Acrocubus octopylus',
 'Acrocubus octopylus ',
 'Artophormis barbadensis',
 'Artophormis dominasinensis',
 'Artophormis gracilis',
 'Calcidiscus leptoporus',
 'Entopyla spp.',
 'Grammatophora spp.',
 'Schematophora obscura',
 'Schematophora speciosa',
 'Top Depth [m]',
 'Top [cm]',
 'Zygodiscus plectopons'}

In [7]:
top

{'Top [cm]'}

In [8]:
top_depth

{'Top Depth [m]'}

In [9]:
bottom_all

{'Bottom Depth [m]', 'Bottom [cm]'}

In [10]:
bottom

{'Bottom [cm]'}

In [11]:
bottom_depth

{'Bottom Depth [m]'}

### Normalize top bottom columns

In [12]:
def normalize_top_bottom(file):
    path = f"{clean_data_path}/{file}"
    content = pd.read_csv(path)
    columns = list(content.columns)
    
    normalized_cols = normalize_columns(top, 'Top [cm]', columns)
    normalized_cols = normalize_columns(bottom, 'Bottom [cm]', normalized_cols)
    normalized_cols = normalize_columns(top_depth, 'Top Depth [m]', normalized_cols)
    normalized_cols = normalize_columns(bottom_depth, 'Bottom Depth [m]', normalized_cols)
    
    changed = columns != normalized_cols
    
    if changed:
        content.columns = normalized_cols
        content = csv_cleanup(content, path)
        content.to_csv(path, index=False)

    return changed

change_columns = [normalize_top_bottom(file) for file in metadata['file']] 

### Update metadata

In [13]:
dict = {"update_top_bottom": change_columns}
new_metadata = update_metadata(metadata, dict)
new_metadata.head()

,file,taxon_group,add_expedition_section_cols,update_sample_col,update_top_bottom
0,363-U1482A-Benthic_Forams.csv,benthic_forams,True,False,False
1,320_U1336A_Nannofossils_2.csv,nannofossils,False,True,False
2,363-U1482A-nannofossils.csv,nannofossils,True,False,False
3,375_U1518F_planktic_forams.csv,planktic_forams,True,False,False
4,320_U1334A_Nannofossils_1.csv,nannofossils,False,True,False


In [14]:
new_metadata.to_csv(metadata_file, index=False)

## Add missing columns

In [3]:
metadata = pd.read_csv(metadata_file)

In [4]:
normalized_columns = [
    'Top [cm]',
    'Bottom [cm]',
    'Top Depth [m]',
    'Bottom Depth [m]', 
    'Sample',
    'Exp',
    'Site',
    'Hole',
    'Core',
    'Type',
    'Section',
    'A/W'
]

In [5]:
change_columns = [add_missing_columns(f"{clean_data_path}/{file}", normalized_columns) for file in metadata['file']] 

In [6]:
dict = {"add_missing_cols": change_columns}
new_metadata = update_metadata(metadata, dict)
new_metadata.head()

,file,taxon_group,add_expedition_section_cols,update_sample_col,update_top_bottom,add_missing_cols
0,363-U1482A-Benthic_Forams.csv,benthic_forams,True,False,False,False
1,320_U1336A_Nannofossils_2.csv,nannofossils,False,True,False,False
2,363-U1482A-nannofossils.csv,nannofossils,True,False,False,False
3,375_U1518F_planktic_forams.csv,planktic_forams,True,False,False,False
4,320_U1334A_Nannofossils_1.csv,nannofossils,False,True,False,False


In [7]:
new_metadata.to_csv(metadata_file, index=False)